In [0]:
%sql
USE CATALOG kaggle_test;
USE SCHEMA bronze;

In [0]:
%python
# Use Managed Identity for authentication
spark.conf.set("fs.azure.account.auth.type.jamesbdatabricksdev.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.jamesbdatabricksdev.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ManagedIdentityTokenProvider")
spark.conf.set("fs.azure.account.oauth2.msi.client.id.jamesbdatabricksdev.dfs.core.windows.net", "jamesbdbconnector")

In [0]:
%sql
CREATE OR REPLACE TABLE stolen_vehicles
AS
SELECT * FROM read_files(
  'abfss://kaggle-test@jamesbdatabricksdev.dfs.core.windows.net/storagepath/bronze/stolen_vehicles.csv',
  format => 'csv',
  header => true,
  mode => 'FAILFAST')

In [0]:
%sql
SELECT * FROM stolen_vehicles LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE vehicle_makes
AS
SELECT * FROM read_files(
  'abfss://kaggle-test@jamesbdatabricksdev.dfs.core.windows.net/storagepath/bronze/make_details.csv',
  format => 'csv',
  header => true,
  mode => 'FAILFAST')

In [0]:
%sql
SELECT * FROM vehicle_makes WHERE make_id = 639;

In [0]:
%sql
CREATE OR REPLACE TABLE make_details_update
AS
SELECT * FROM read_files(
  'abfss://kaggle-test@jamesbdatabricksdev.dfs.core.windows.net/storagepath/bronze/make_details_update.csv',
  format => 'csv',
  header => true,
  mode => 'FAILFAST')

In [0]:
%sql
SELECT * FROM make_details_update

In [0]:
%sql
MERGE INTO vehicle_makes
USING make_details_update
ON vehicle_makes.make_id = make_details_update.make_id
WHEN MATCHED THEN
UPDATE SET *
WHEN NOT MATCHED THEN
INSERT *;

In [0]:
%sql
DESCRIBE HISTORY vehicle_makes;